# Let's Build a Quant Trading Strategy - Part 3

In [772]:
# y_hat = model(x)
# orders = strategy(y_hat)
# execute(orders)

In [773]:
# 1st video = Research
# 2nd video = Strategy
# 3rd video = Implementation

# I strongly recommend to watch the first and second video but I try to make them independent by recapping

### Goals

1. Code it all together and put it live with real money.
2. Show you how to build the architecture of a trading system 
3. Learn how to stream exchange data efficiently and react to events.

In [774]:
# By the end of this video, we will have coded everything all together in Python and put it live!
# Ideally I would prefer to write this in Rust but we will stick to Python. Maybe if you would like a Rust version, please leave a comment,
# Put it live with real money and see what happens.
# I will do a follow up video to see how it performs. 
# Even if it doesn't perform as expected, we can learn from it. 
# We might experience model drift - where where the pattern it learned starts to no longer hold true. 
# You might hear this be called alpha decay - which is your profits decay because of model drift. 
# So even if the pattern lose persistence; we can learn from it. For example, using sliding window instead and retrain.


In [775]:
# This will video will be more software engineering now as we have to code it all together.

# 1. Ability to stream data from the exchange efficiently - just like Netflix - 
# Netflix intelligently buffers the data - you never have to wait for Netflix to download the next scenes - it's all seamless
# Our system needs to stream data seamlessly
# Our system needs to react to the prices - events
# Our system needs to handle events


## Recap

### Part 1: Research

In [776]:
import models
import torch
import research

model = models.LinearModel(3)
# security alert 
model.load_state_dict(torch.load('model_weights.pth', weights_only=True))
model.eval()

LinearModel(
  (linear): Linear(in_features=3, out_features=1, bias=True)
)

### AR(3) Model - 12h forecast horizon

In [777]:
research.print_model_params(model)

linear.weight:
[[-0.10395038 -0.06726477  0.02827305]]
linear.bias:
[0.00067121]


### Part 2: Strategy Recap

In [778]:
# Showed a strategy that generates ~14% without any optimization.
# I then showed 2 strategy decisions that turned that the strategy 14% to >40%
# The reason for the huge increase in returns were a combination:
# 1. Model's statistical edge 
# 2. Compounding Trade Sizing 
# 3. Leverage



## The fundamental building block: Tick

In [779]:
from abc import ABC, abstractmethod
from typing import Generic, TypeVar

T = TypeVar('T')  # input type
R = TypeVar('R')  # output type

class Tick(ABC, Generic[T, R]):
    @abstractmethod
    def on_tick(self, val: T) -> R:
        """Handle a new tick and optionally return a result."""
        pass

## The fundamental data structure: Window

In [780]:
from collections import deque
from typing import Deque, Optional
import numpy as np

class DequeWindow(Tick[T, Optional[T]], Generic[T]):
    def __init__(self, n: int):
        self._data: Deque[T] = deque(maxlen=n)

    def append_left(self, val: T) -> Optional[T]:
        dropped = None
        if self.is_full():
            dropped = self._data[-1]
        self._data.appendleft(val)
        return dropped

    def on_tick(self, val: T) -> Optional[T]:
        """Append a value and return the oldest value dropped (if any)."""
        dropped = None
        if len(self._data) == self.capacity():
            dropped = self._data[0]
        self._data.append(val)
        return dropped

    def capacity(self) -> int:
        n = self._data.maxlen
        return 0 if n is None else n
    
    def is_full(self) -> bool:
        return self._data.maxlen == len(self._data)
    
    def to_numpy(self) -> np.ndarray:
        return np.array(self._data)
    
    def __repr__(self) -> str:
        cls_name = self.__class__.__name__
        return f"{cls_name}(capacity={self._data.maxlen}, values={list(self._data)})"

### Streaming Example: Sliding Window

In [781]:
w = DequeWindow(3)
w.on_tick(1)
w

DequeWindow(capacity=3, values=[1])

In [782]:
w = DequeWindow(3)
w.on_tick(1)
w.on_tick(2)
w

DequeWindow(capacity=3, values=[1, 2])

In [783]:
w = DequeWindow(3)
w.on_tick(1)
w.on_tick(2)
w.on_tick(3)
w


DequeWindow(capacity=3, values=[1, 2, 3])

In [784]:
w = DequeWindow(3)
w.on_tick(1)
w.on_tick(2)
w.on_tick(3)
old_v = w.on_tick(4)

(old_v, w)

(1, DequeWindow(capacity=3, values=[2, 3, 4]))

In [785]:
w = DequeWindow(3)
w.on_tick(1)
w.on_tick(2)
w.on_tick(3)
w.on_tick(4)
old_v = w.on_tick(5)

(old_v, w)

(2, DequeWindow(capacity=3, values=[3, 4, 5]))

### Array-based Window

In [786]:
import numpy as np
from typing import Optional

class NumpyWindow(Tick[T, Optional[T]]):
    """Fixed-capacity sliding window for scalar values using a NumPy array (shift on append)."""

    def __init__(self, n: int, dtype=np.float64):
        if n <= 0:
            raise ValueError("Capacity must be positive.")
        self._capacity = n
        self._data = np.zeros(n, dtype=dtype)
        self._size = 0

    def on_tick(self, val: float) -> Optional[float]:
        """
        Append a scalar value and return the oldest value dropped (if any).
        Shifts elements in-place without copying slices.
        """
        dropped = None

        if self._size < self._capacity:
            self._data[self._size] = val
            self._size += 1
        else:
            dropped = self._data[0]
            # shift left in-place
            for i in range(1, self._capacity):
                self._data[i - 1] = self._data[i]
            self._data[-1] = val

        return dropped


    def __getitem__(self, idx: int) -> float:
        """Index access (0 = oldest)."""
        if not 0 <= idx < self._size:
            raise IndexError("Index out of range.")
        return self._data[idx]

    def __len__(self) -> int:
        return self._size

    def capacity(self) -> int:
        return self._capacity

    def is_full(self) -> bool:
        return self._size == self._capacity

    def values(self) -> np.ndarray:
        """Return current valid values as a contiguous 1D array (no copy needed)."""
        return self._data[:self._size]

    def __repr__(self) -> str:
        vals = self.values().tolist()
        return f"{self.__class__.__name__}(capacity={self._capacity}, size={self._size}, values={vals})"


### Streaming Example: Numpy Sliding Window

In [787]:
w = NumpyWindow(3)
w.on_tick(1)
w

NumpyWindow(capacity=3, size=1, values=[1.0])

In [788]:
w = NumpyWindow(3)
w.on_tick(1)
w.on_tick(2)
w

NumpyWindow(capacity=3, size=2, values=[1.0, 2.0])

In [789]:
w = NumpyWindow(3)
w.on_tick(1)
w.on_tick(2)
w.on_tick(3)
w

NumpyWindow(capacity=3, size=3, values=[1.0, 2.0, 3.0])

In [790]:
w = NumpyWindow(3)
w.on_tick(1)
w.on_tick(2)
w.on_tick(3)
v = w.on_tick(4)
(v, w)

(np.float64(1.0), NumpyWindow(capacity=3, size=3, values=[2.0, 3.0, 4.0]))

### Benchmark Numpy Window vs Deque Window

In [791]:
def benchmark_window(window, n):
    for i in range(n):
        window.on_tick(i)

window_size = 3
n = 10000000

In [792]:
%%time
benchmark_window(NumpyWindow(window_size), n)

CPU times: user 4.69 s, sys: 23.8 ms, total: 4.71 s
Wall time: 4.74 s


In [793]:
%%time
benchmark_window(DequeWindow(window_size), n)

CPU times: user 1.12 s, sys: 5.02 ms, total: 1.13 s
Wall time: 1.13 s


### Streaming the Last Known Value (To Track Close Price)

In [794]:
class Last(Tick[T, T], Generic[T]):
    def __init__(self):
        self._value: Optional[T] = None

    def on_tick(self, val: T) -> Optional[T]:
        """Append a value and return the oldest value dropped (if any)."""
        self._value = val
        return val

    def __repr__(self) -> str:
        cls_name = self.__class__.__name__
        return f"{cls_name}(value={self._value})"

### Example: Streaming Last Value

In [795]:
last_val = Last()
last_val

Last(value=None)

In [796]:
last_val = Last()
last_val.on_tick(1)
last_val.on_tick(2)
last_val.on_tick(3)
last_val

Last(value=3)

### Streaming Log Returns

#### Quick Log Returns Recap

In [797]:
portfolio_values = [100, 120, 100]
log_returns = [
    np.log(portfolio_values[1]/portfolio_values[0]), 
    np.log(portfolio_values[2]/portfolio_values[1]), 
]
log_returns

[np.float64(0.1823215567939546), np.float64(-0.1823215567939546)]

In [798]:
np.sum(log_returns)

np.float64(0.0)

In [799]:
import numpy as np

class LogReturn(Tick[float, float], Generic[T]):
    def __init__(self):
        self._window = NumpyWindow(2)

    def on_tick(self, val: float) -> Optional[float]:
        self._window.on_tick(val)
        if self._window.is_full():
            return np.log(self._window[1] / self._window[0])
        else:
            return None
        
    def __repr__(self) -> str:
        cls_name = self.__class__.__name__
        return f"{cls_name}(window={self._window})"
    

### Streaming Example: Log Returns

In [800]:
f = LogReturn()
f.on_tick(90.0)
f

LogReturn(window=NumpyWindow(capacity=2, size=1, values=[90.0]))

In [801]:
f = LogReturn()
f.on_tick(90.0)
v = f.on_tick(100.0)
(v, f)

(np.float64(0.10536051565782635),
 LogReturn(window=NumpyWindow(capacity=2, size=2, values=[90.0, 100.0])))

In [802]:
np.log(100/90)

np.float64(0.10536051565782635)

In [803]:
f = LogReturn()
f.on_tick(90.0)
f.on_tick(100.0)
v = f.on_tick(150.0)
(v, f)

(np.float64(0.4054651081081644),
 LogReturn(window=NumpyWindow(capacity=2, size=2, values=[100.0, 150.0])))

In [804]:
np.log(150/100)

np.float64(0.4054651081081644)

### Streaming Auto-Regressive Log Returns Lags

#### Auto-Regression Recap

In [805]:
time_series = [0.1, -0.2, -0.3]
lag_1 = time_series[-2]
lag_1

-0.2

In [806]:
lag_2 = time_series[-3]
lag_2

0.1

In [807]:

class LogReturnLags(Tick[float, torch.Tensor]):
    def __init__(self, no_lags: int):
        self._lags = DequeWindow(no_lags)
        self._log_return = LogReturn()
    
    def on_tick(self, val: float) -> torch.Tensor | None:
        log_ret = self._log_return.on_tick(val)
        if log_ret is not None:
            self._lags.append_left(log_ret)
            return torch.tensor(self._lags.to_numpy(), dtype=torch.float32) if self._lags.is_full() else None
        else:
            return None
        
    def __repr__(self) -> str:
        cls_name = self.__class__.__name__
        return f"{cls_name}(lags={self._lags}, log_return={self._log_return})" 

### Streaming Example: Auto-Regressive Log Returns

In [808]:
lags = LogReturnLags(3)
v = lags.on_tick(90)
lags

LogReturnLags(lags=DequeWindow(capacity=3, values=[]), log_return=LogReturn(window=NumpyWindow(capacity=2, size=1, values=[90.0])))

In [809]:
lags = LogReturnLags(3)
lags.on_tick(90)
lags.on_tick(100)
lags


LogReturnLags(lags=DequeWindow(capacity=3, values=[np.float64(0.10536051565782635)]), log_return=LogReturn(window=NumpyWindow(capacity=2, size=2, values=[90.0, 100.0])))

In [810]:
lags = LogReturnLags(3)
lags.on_tick(90)
lags.on_tick(100)
lags.on_tick(150)
lags

LogReturnLags(lags=DequeWindow(capacity=3, values=[np.float64(0.4054651081081644), np.float64(0.10536051565782635)]), log_return=LogReturn(window=NumpyWindow(capacity=2, size=2, values=[100.0, 150.0])))

In [811]:
lags = LogReturnLags(3)
lags.on_tick(90)
lags.on_tick(100)
lags.on_tick(150)
lags.on_tick(110)

tensor([-0.3102,  0.4055,  0.1054])

In [812]:
[
    np.log(110/150),
    np.log(150/100),
    np.log(100/90),
]

[np.float64(-0.3101549283038396),
 np.float64(0.4054651081081644),
 np.float64(0.10536051565782635)]

In [813]:
lags = LogReturnLags(3)
lags.on_tick(90)
lags.on_tick(100)
lags.on_tick(150)
lags.on_tick(110)
features = lags.on_tick(160)
features

tensor([ 0.3747, -0.3102,  0.4055])

### Example: Streaming the Features into our Model 

In [814]:
import models
model = models.LinearModel(3)
# security alert 
model.load_state_dict(torch.load('model_weights.pth', weights_only=True))
model.eval()

LinearModel(
  (linear): Linear(in_features=3, out_features=1, bias=True)
)

In [815]:
X = features
y_hat = model(X)
y_hat

tensor([-0.0060], grad_fn=<ViewBackward0>)

In [816]:
y_hat[0]

tensor(-0.0060, grad_fn=<SelectBackward0>)

### Build The Trading System

In [817]:
from decimal import Decimal
from typing import Dict, List
from dataclasses import dataclass
import torch.nn as nn
import polars as pl

def decimal_sign(d: Decimal) -> int:
    return 1 if d > Decimal(0) else -1

def is_long(x: Decimal) -> bool:
    return decimal_sign(x) > 0

@dataclass(frozen=True)
class Order:
    sym: str
    signed_qty: Decimal

    def __str__(self) -> str:
        sign = "LONG" if self.signed_qty > 0 else "SHORT"
        return f"Order({sign} {self.signed_qty} {self.sym})"

@dataclass(frozen=True)
class Trade:
    sym: str
    qty: Decimal
    price: Decimal
    is_long: bool

    def __str__(self) -> str:
        sign = "LONG" if self.signed_qty > 0 else "SHORT"
        return f"Trade({sign} {self.qty} {self.sym} {self.price} {self.is_long})"

    def signed_qty(self) -> Decimal:
        return 1 if self.is_long else -1 * self.qty

@dataclass
class Position:
    sym: str
    signed_qty: Decimal
    price: Decimal

    def close(self) -> "Order":
        return Order(self.sym, -self.signed_qty)
    
    def is_long(self) -> bool:
        return is_long(self.signed_qty)


### Create Base Class for Market

In [818]:
from abc import ABC, abstractmethod
from decimal import Decimal
from typing import Dict

class Account(ABC):
    """Abstract base class representing a trading account."""

    @abstractmethod
    def balance(self) -> Decimal:
        """Return the current account balance."""
        pass

    @abstractmethod
    def market_order(self, sym: str, qty: Decimal, price: Decimal = None) -> "Trade":
        """Execute a market order and return a Trade result."""
        pass

    @abstractmethod
    def get_position(self, sym: str) -> Dict[str, "Position"]:
        """Return current open positions."""
        pass    




### Create Test Account

In [819]:
from decimal import Decimal
from typing import Dict, Optional

class TestAccount(Account):
    """A simulated account for testing or paper trading."""

    def __init__(self, time_series: pl.DataFrame, _balance: Decimal) -> None:
        self._balance = _balance
        self.time_series = time_series
        self._positions: Dict[str, Trade] = {}
        self.counter: int = 0
        self.trades: List[Trade] = [] 

    def balance(self) -> Decimal:
        return self._balance

    def get_trade_price(self) -> Decimal:
        price = self.time_series["close"][self.counter]
        self.counter += 1
        return Decimal(price)

    def market_order(self, sym: str, signed_qty: Decimal, price: Decimal = None) -> "Trade":
        # Simulate a fill at fixed price
        if price is None:
            price = self.get_trade_price()
        
        signed_val = signed_qty * price
        # Update balance and position
        trade = Trade(sym=sym, qty=signed_qty, price=price, is_long=is_long(signed_qty))
        pnl = self._update_position(sym, trade)
        if pnl:
            self._balance += pnl
        
        self.trades.append(trade)
        print(f"trades = {self.trades}")
        print(f"positions = {self._positions}")

    def get_position(self, sym) -> Optional[Position]:
        return self._positions.get(sym)
    
    def _update_position(self, sym: str, trade: Trade) -> Optional[Decimal]:
        position = self._positions.pop(sym, None)
        pnl = None
        if position:
            is_trade_long = trade.is_long
            is_pos_long = position.is_long
            if is_pos_long and not is_trade_long:
                entry_val = position.price * trade.signed_qty()
                exit_val = trade.price * trade.signed_qty()                
                pnl = exit_val - entry_val
            else:
                raise "expected closing trade"
        else:
            self._positions[sym] = trade
        return pnl

    def __repr__(self) -> str:
        return f"TestAccount(balance={self._balance}, positions={self._positions})"


### Example: Market Orders in Test Account

In [820]:
time_series = pl.read_csv('BTCUSDT_12h_ohlc.csv', try_parse_dates=True)
time_series

datetime,open,high,low,close
datetime[μs],f64,f64,f64,f64
2024-10-29 00:00:00,69939.9,71607.0,69733.0,71440.1
2024-10-29 12:00:00,71440.0,73660.0,70900.0,72739.5
2024-10-30 00:00:00,72739.5,72797.4,71931.1,71995.0
2024-10-30 12:00:00,71994.9,72984.9,71444.2,72349.0
2024-10-31 00:00:00,72349.0,72720.3,72030.5,72213.3
…,…,…,…,…
2025-10-07 12:00:00,124397.1,125098.0,120516.0,121286.5
2025-10-08 00:00:00,121286.6,123150.0,121005.3,122825.7
2025-10-08 12:00:00,122825.8,124170.6,121607.8,123237.5


In [821]:
time_series['close']

close
f64
71440.1
72739.5
71995.0
72349.0
72213.3
…
121286.5
122825.7
123237.5


In [822]:
time_series['close'][0]

71440.1

In [823]:
100.0/71440.1

0.0013997740764640585

In [824]:
# place market order
acc = TestAccount(time_series, Decimal(100.0))

In [825]:
price = Decimal(100)
trade_value = Decimal(100)
qty = Decimal(1.0)
acc.market_order('BTCUSDT', qty, Decimal(price))

trades = [Trade(sym='BTCUSDT', qty=Decimal('1'), price=Decimal('100'), is_long=True)]
positions = {'BTCUSDT': Trade(sym='BTCUSDT', qty=Decimal('1'), price=Decimal('100'), is_long=True)}


In [826]:
acc

TestAccount(balance=100, positions={'BTCUSDT': Trade(sym='BTCUSDT', qty=Decimal('1'), price=Decimal('100'), is_long=True)})

In [827]:
price = 110.0
acc.market_order('BTCUSDT', -qty, Decimal(price))

trades = [Trade(sym='BTCUSDT', qty=Decimal('1'), price=Decimal('100'), is_long=True), Trade(sym='BTCUSDT', qty=Decimal('-1'), price=Decimal('110'), is_long=False)]
positions = {}


In [828]:
-12/112

-0.10714285714285714

In [829]:
(1 - 0.10714285714285714) * 110

98.21428571428572

In [830]:
price = Decimal(112.0)
qty = acc.balance() / price 
acc.market_order('BTCUSDT', qty, Decimal(price))

trades = [Trade(sym='BTCUSDT', qty=Decimal('1'), price=Decimal('100'), is_long=True), Trade(sym='BTCUSDT', qty=Decimal('-1'), price=Decimal('110'), is_long=False), Trade(sym='BTCUSDT', qty=Decimal('0.9821428571428571428571428571'), price=Decimal('112'), is_long=True)]
positions = {'BTCUSDT': Trade(sym='BTCUSDT', qty=Decimal('0.9821428571428571428571428571'), price=Decimal('112'), is_long=True)}


In [831]:
price = Decimal(100.0)
acc.market_order('BTCUSDT', -qty, Decimal(price))

trades = [Trade(sym='BTCUSDT', qty=Decimal('1'), price=Decimal('100'), is_long=True), Trade(sym='BTCUSDT', qty=Decimal('-1'), price=Decimal('110'), is_long=False), Trade(sym='BTCUSDT', qty=Decimal('0.9821428571428571428571428571'), price=Decimal('112'), is_long=True), Trade(sym='BTCUSDT', qty=Decimal('-0.9821428571428571428571428571'), price=Decimal('100'), is_long=False)]
positions = {}


### Develop Strategy API

In [ ]:
class Strategy(ABC):
    """Abstract base class representing a trading account."""

    @abstractmethod
    def on_tick(self, price: float, account: Account) -> List[Order] | None:
        pass

class BasicTakerStrat(Strategy):
    def __init__(self, 
                 sym: str,
                 model: nn.Module, 
                 log_return_lags: LogReturnLags, 
                 scale_factor: Decimal = None) -> None:
        self.sym = sym
        self.model = model
        self.log_return_lags = log_return_lags
        if scale_factor is None:
            scale_factor = Decimal(1.0)
        self.scale_factor = Decimal(scale_factor)

    def _signed_trade_size(self, y_hat: float, account: Account) -> Decimal:
        dir_signal = np.sign(y_hat)
        balance =  account.balance()
        print(f"balance = {balance}")
        v = Decimal(dir_signal) * balance
        return v * self.scale_factor

    def _create_orders(self, y_hat: torch.Tensor, account: Account) -> List[Order]:
        print(f"account = {account}")
        print(f"y_hat = {y_hat}")
        not_val = self._signed_trade_size(y_hat.item(), account)
        print(f"notional value = {not_val}")
        open_order = Order(self.sym, not_val)
        print(f"open order = {open_order}")
        
        position = account.get_position(self.sym)
        
        if position:
            close_order = Order(position.sym, -position.signed_qty)
            return [close_order, open_order]
        return [open_order]      

    def on_tick(self, price: float, acc: Account) -> List[Order] | None:
        X = self.log_return_lags.on_tick(price)
        if X is not None:
            with torch.no_grad():
                print(f"X = {X}")
                y_hat = self.model(X)
                orders = self._create_orders(y_hat, acc)
                return orders
        return None

In [833]:
sym = 'BTCUSDT'
lags = LogReturnLags(3)
acc = TestAccount(time_series, Decimal(100.0))

strat = BasicTakerStrat(sym, model, lags, Decimal(1.0))
strat.on_tick(100.0, acc)
strat.on_tick(110.0, acc)
strat.on_tick(90.0, acc)
orders = strat.on_tick(105, acc)
orders

X = tensor([ 0.1542, -0.2007,  0.0953])
account = TestAccount(balance=100, positions={})
y_hat = tensor([0.0008])
balance = 100
notional value = 100
open order = Order(LONG 100 BTCUSDT)


UnboundLocalError: cannot access local variable 'orders' where it is not associated with a value

In [ ]:
x = [np.log(105/90), np.log(90/110.0), np.log(110/100)]
x

In [ ]:
orders = strat.on_tick(115, acc)
orders

In [ ]:
import research
research.print_model_params(model)

In [ ]:
w = np.array([-0.10395038, -0.06726477,  0.02827305]) 
w

In [ ]:
b = 0.00067121
np.dot(w,x) + b

In [ ]:
lags = LogReturnLags(3)
acc = TestAccount(time_series, Decimal(100.0))

strat = BasicTakerStrat(sym, model, lags, Decimal(1.0))
strat.on_tick(100.0, acc)
strat.on_tick(110.0, acc)
strat.on_tick(90.0, acc)
orders = strat.on_tick(105, acc)
order = orders[0]
order
acc.market_order(sym, order.signed_val, Decimal(price))
# strat.on_tick(120, acc)

### Time Interval Timer - 12h

In [ ]:
from datetime import datetime, timezone

def epoch_now():
    return datetime.now(timezone.utc).timestamp()

epoch_utc = epoch_now()
epoch_utc

In [ ]:
def ts_bar(epoch: float | int, n: int) -> int:
    epoch = int(epoch)
    return epoch - (epoch % n)

In [ ]:
min_in_secs = 60

In [ ]:
ts_bar(epoch_utc, min_in_secs)

In [ ]:
ts_bar(epoch_utc + 61, min_in_secs)

In [ ]:
class IntervalTimer(Tick[int, bool]):
    def __init__(self, interval_mins: int):
        self.last_bar: int = 0  
        self.time_interval = interval_mins * 60

    def on_tick(self, epoch: int) -> bool:
        bar = ts_bar(epoch, self.time_interval)
        if bar > self.last_bar:
            self.last_bar = bar
            return True
        return False

### Example: Timer

In [ ]:
timer = IntervalTimer(1)

now = epoch_now()

timer.on_tick(now)

In [ ]:
timer.on_tick(now + 2)

In [ ]:
timer.on_tick(now + 60)

In [ ]:
timer.on_tick(now + 60)

### Execution

In [ ]:
# y_hat = model(x)
# orders = strategy(y_hat)
# execute(orders)

In [ ]:
import asyncio

class TestExecutionEngine:
    def __init__(self, time_series: pl.DataFrame, strat: Strategy, timer: IntervalTimer, account: TestAccount):
        self.time_series = time_series
        self.strat: Strategy = strat
        self.timer: IntervalTimer = timer
        self.account: TestAccount = account
        self.counter: int = 0
    
    async def _execute(self, orders: List["Order"]):
        """Execute orders asynchronously (can simulate latency, slippage, etc)."""
        
        if not orders:
            await asyncio.sleep(0.01)
        return


    async def start(self, sleep_secs = None):
        """Run main event loop asynchronously."""
        print("Execution engine started...")
        while True:
            # now = epoch_now()

            # if not self.timer.on_tick(now):
            #     await asyncio.sleep(0.01)
            #     continue
            if sleep_secs:
                await asyncio.sleep(5.0)

            if self.counter >= len(self.time_series):
                return
            
            dt = self.time_series['datetime'][self.counter]
            close_price = float(self.time_series["close"][self.counter])
            
            self.counter += 1
            orders = self.strat.on_tick(close_price, self.account)

            if orders and len(orders) > 0:
                print(f'{dt}: {close_price} = {orders}')
            else:
                print(f'{dt}: {close_price}')
            

            await self._execute(orders)
    



In [ ]:
import asyncio

def build_strat() -> BasicTakerStrat:
    sym = 'BTCUSDT'
    lags = LogReturnLags(3)
    return BasicTakerStrat(sym, model, lags, 1.0)

time_series = pl.read_csv('BTCUSDT_12h_ohlc.csv', try_parse_dates=True)

account = TestAccount(time_series, Decimal(100.0))
strat = build_strat()
engine = TestExecutionEngine(time_series, strat, timer, acc)

async def main():
    print("Start...")

    await engine.start(sleep_secs=2)
    print("Done!")

# You can directly await
await main()